# 16 1 5

Adding in expected secondary position, which will be used to find the observed secondary lobe closest to the expected secondary position. 

PS3 takes in the following data fields: 
- *TID* — Target ID. If the Target ID is a “D”, then PS3 will process this file, otherwise it will be ignored.
This allows a master database to contain, for instance, single stars used for deconvolution, slit mask
or drift calibrations, etc.
- *DSFN* — Double star (FITS cube) file name.
- *RSFN* — Reference star (FITS) cube file name.
- *ThetaC* — Calculated (input) estimated, predicted, or last catalog reported double star position angle.
- *RhoC* — Calculated (input) estimated, predicted, or last catalog reported double star separation.
- *Delta* — Camera Angle (degrees). Camera orientation angle with respect to the sky.
- *E* — Plate scale (arc seconds / pixel).

For all the FITS data I am working with from April 2014 Kitt Peak run:
- *Delta* = 0 [degrees]
- *E* = 0.01166 [arc seconds / pixel]


In [ ]:
# Using calculated centroid data from end of previous notebook along 
#  with calibration and expected position data from run log to calculate 
#  observed secondary location.

In [ ]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2,ifft2,fftshift
from speckle_fns import fits_import, circ_filter1
from speckle_fns import deconv0,deconv1, postprocess
from speckle_fns import filter_lpf, filter_hpf, filter_interference
from speckle_fns import fits_view
from collections import namedtuple
import sys
import cv2
import math

%matplotlib inline

In [ ]:
# Centroid values from end of 15-12-28 notebook:
centroids = np.zeros((5,2,2))

centroids[0] = [[ 403.24772626, 405.91946966],[ 108.75227374, 106.08053034]]
centroids[1] = [[ 220.00967484, 255.04554408],[ 291.99032516, 256.95445592]]
centroids[2] = [[ 469.02557551, 506.57034716],[ 42.97582522, 5.06646098]]
centroids[3] = [[ 225.11283002, 274.88879163],[ 286.88716998, 237.11120837]]
centroids[4] = [[ 282.11470187, 275.93283984],[ 229.88529813, 236.06716016]]

# Expected positions of each pair from "KP 4-14 Run Log.xlsx"
thetaCs = np.array((204,63,32,305,124))
rhoCs = np.array((44.7,4.9,0.3,0.5,0.4))

# Calibration values from "KP 4-14 Run Log.xlsx"
delta = 0
E = 0.01166

In [ ]:
# Select which set of data to work with

k = 3

centroid = centroids[k]

In [ ]:
# Functions & structs used in this file

# convert float pixel indices to rounded, int16 values
def pixelRound(val):
    return (np.round(val)).astype(np.int16)

# position class is for holding the position of anything
#  Position may be represented in Cartesian or Polar coordinates
#  Want to be able to store both kinda of coordinates
class position:
    def __init__(self, midpoint=None, theta=None, rho=None, x=None, y=None):
        self.theta = theta
        self.rho = rho
        self.x = x
        self.y = y
        self.midpoint = midpoint
        
    # Orbital plot has 0deg pointing down on the screen [resulting in 90deg shift]
    # Theta rotates counterclockwise from 0deg
    # Positive pixel direction is downward [resulting in Y phase reversal]
    # Polar coordinates centered on middle pixel of image
        
    # convert theta and rho values to image x and y coordinates    
    def polar2cart(self):
        self.x = self.midpoint + (self.rho * np.cos(np.radians(self.theta-90)))
        self.y = self.midpoint - (self.rho * np.sin(np.radians(self.theta-90)))
        
    # convert x and y coordinates to theta and rho values
    def cart2polar(self):
        # Get cartesians centered on 0 for calculations
        x = self.x-self.midpoint
        y = self.midpoint-self.y
        
        self.rho = np.linalg.norm((x,y))
        self.theta = np.degrees(np.arctan2(y,x))+90
        # Add 360 deg if theta is a negative angle
        self.theta = self.theta + 360*(self.theta<0) 
        
# star class is for holding position of a star
#  A star has coordinates in camera (in pixels) and also
#  in the observed sky (in arcseconds). Theta represents 
#  rotation of camera in relation to the sky
#starType = namedtuple('starType',"cam sky error")
class star:
    def __init__(self, midpoint = None, E=None, delta=None):
        self.cam = position(midpoint = midpoint)
        self.sky = position(midpoint = midpoint)
        self.E = E
        self.delta = delta
    
        
    # Convert from polar coords in sky to camera
    def sky2cam(self):
        self.cam.theta = self.sky.theta+self.delta
        self.cam.rho =   self.sky.rho/self.E
        
    # Convert from polar coords in camera to sky    
    def cam2sky(self):
        self.sky.theta = self.cam.theta-self.delta
        self.sky.rho   = self.cam.rho*self.E

# Circular marking for noting locations of objects
marker = namedtuple('marker',"stroke radius color")
centroidMarker = marker(stroke=1,radius=10,color=(255,0,0))
expectedMarker = marker(stroke=1,radius=10,color=(0,255,0))
observedMarker = marker(stroke=1,radius=5,color=(0,0,255))

In [ ]:
## Displaying calculated centroid values along with expected secondary star locations

# Create blank image
imgDim = 512
img = np.zeros((imgDim,imgDim,3), np.uint8)

# Object for the expected star position
expectedPos = star(midpoint = imgDim/2, E = E, delta = delta)
# Object for the closest centroid to expected star position
observedPos = star(midpoint = imgDim/2, E = E, delta = delta)

# Assigning position values from previous data
expectedPos.sky.theta = thetaCs[k]
expectedPos.sky.rho = rhoCs[k]

# Calculate camera locations of expected star pos
expectedPos.sky2cam()

# Converting to cartesian
expectedPos.cam.polar2cart()

# Mark centroid locations on image
cv2.circle(img, (centroid[0,0].astype(np.uint16),centroid[0,1].astype(np.uint16)), centroidMarker.radius, centroidMarker.color, centroidMarker.stroke)
cv2.circle(img, (centroid[1,0].astype(np.uint16),centroid[1,1].astype(np.uint16)), centroidMarker.radius, centroidMarker.color, centroidMarker.stroke)

# Mark expected star location on image
cv2.circle(img, (pixelRound(expectedPos.cam.x),pixelRound(expectedPos.cam.y)),expectedMarker.radius, expectedMarker.color, expectedMarker.stroke)

plt.figure(figsize=(10,10))
plt.imshow(img)

# 16 1 6

Testing methods of classes

# 16 1 7

Finding centroids closest to expected location

Finding errors between expected and closest centroid

In [ ]:
# Finding closest centroid

# Initialize distance list
distance = [math.inf,math.inf]

# Check distance to each centroid
for i in np.arange(2):
    distance[i] = np.linalg.norm((expectedPos.cam.x-centroid[i,0],expectedPos.cam.y-centroid[i,1]))

# Find index of minimum distance    
index = (distance.index(min(distance)))

# Centroid with minimum distance becomes the observed position
(observedPos.cam.x,observedPos.cam.y) = (centroid[index,0],centroid[index,1])

# Create Blank Image
img = np.zeros((imgDim,imgDim,3), np.uint8)

# Mark centroid locations on image
cv2.circle(img, (centroid[0,0].astype(np.uint16),centroid[0,1].astype(np.uint16)), centroidMarker.radius, centroidMarker.color, centroidMarker.stroke)
cv2.circle(img, (centroid[1,0].astype(np.uint16),centroid[1,1].astype(np.uint16)), centroidMarker.radius, centroidMarker.color, centroidMarker.stroke)

# Mark expected star location on image
cv2.circle(img, (pixelRound(expectedPos.cam.x),pixelRound(expectedPos.cam.y)),expectedMarker.radius, expectedMarker.color, expectedMarker.stroke)

# Mark observed star location on image
cv2.circle(img, (pixelRound(observedPos.cam.x),pixelRound(observedPos.cam.y)),observedMarker.radius, observedMarker.color, observedMarker.stroke)

plt.figure(figsize=(10,10))
plt.imshow(img)

In [ ]:
# Calculating percent errors in position

# Calculate polar coordinates of observed star
observedPos.cam.cart2polar()

# Calculate error in radius
errorRadius = observedPos.cam.rho-expectedPos.cam.rho

# Calculate error in angle
errorTheta  = observedPos.cam.theta-expectedPos.cam.theta
while(errorTheta<-180): # Get angle between -180 to 180
    errorTheta += 360
while(errorTheta>180):  # Get angle between -180 to 180
    errorTheta -= 360

# Calculate error in cartesian distance
errorDistance = distance[index]
                          
print(errorRadius)                 
print(errorTheta)
print(errorDistance)

In [ ]:
# Calculating astrometric data of observed stars
print(observedPos.cam.rho, observedPos.cam.theta)
# Convert camera view of 
observedPos.cam2sky()

# View astrometric data of observed stars
print(expectedPos.sky.rho, expectedPos.sky.theta)
print(observedPos.sky.rho, observedPos.sky.theta)